## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.layers import ReLU
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

C:\Users\walter\Anaconda3\envs\DLTest\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\walter\Anaconda3\envs\DLTest\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
classifier=Sequential()

#卷積組合
classifier.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', input_shape=(32,32,3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(ReLU())
classifier.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))


'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Conv2D(filters=64, kernel_size=(3,3), padding='same'))
classifier.add(BatchNormalization())
classifier.add(ReLU())

classifier.add(Conv2D(filters=64, kernel_size=(3,3), padding='same'))
classifier.add(BatchNormalization())
classifier.add(ReLU())



classifier.add(Conv2D(filters=128, kernel_size=(3,3), padding='same'))
classifier.add(BatchNormalization())
classifier.add(ReLU())

classifier.add(Conv2D(filters=128, kernel_size=(3,3), padding='same'))
classifier.add(BatchNormalization())
classifier.add(ReLU())

classifier.add(Conv2D(filters=128, kernel_size=(3,3), padding='same'))
classifier.add(BatchNormalization())
classifier.add(ReLU())

classifier.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

classifier.add(Conv2D(filters=256, kernel_size=(3,3), padding='same'))
classifier.add(BatchNormalization())
classifier.add(ReLU())

classifier.add(Conv2D(filters=256, kernel_size=(3,3), padding='same'))
classifier.add(BatchNormalization())
classifier.add(ReLU())

classifier.add(Conv2D(filters=512, kernel_size=(3,3), padding='same'))
classifier.add(BatchNormalization())
classifier.add(ReLU())

#flatten
#classifier.add(Flatten())
classifier.add(GlobalAveragePooling2D())

#FC
#classifier.add(Dense(units= 1024, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))
classifier.summary()

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=64,epochs=100)

W0309 15:12:18.041326 15444 deprecation_wrapper.py:119] From C:\Users\walter\Anaconda3\envs\DLTest\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0309 15:12:18.070223 15444 deprecation_wrapper.py:119] From C:\Users\walter\Anaconda3\envs\DLTest\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0309 15:12:18.075232 15444 deprecation_wrapper.py:119] From C:\Users\walter\Anaconda3\envs\DLTest\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0309 15:12:18.108122 15444 deprecation_wrapper.py:119] From C:\Users\walter\Anaconda3\envs\DLTest\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W030

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
re_lu_2 (ReLU)               (None, 16, 16, 64)        0         
__________

50000/50000 [==============================] - 18s 362us/step - loss: 0.0282 - acc: 0.9899
Epoch 37/100
50000/50000 [==============================] - 18s 360us/step - loss: 0.0343 - acc: 0.9883
Epoch 38/100
50000/50000 [==============================] - 18s 362us/step - loss: 0.0249 - acc: 0.9915
Epoch 39/100
50000/50000 [==============================] - 18s 360us/step - loss: 0.0215 - acc: 0.9924
Epoch 40/100
50000/50000 [==============================] - 18s 360us/step - loss: 0.0314 - acc: 0.9895
Epoch 41/100
50000/50000 [==============================] - 18s 360us/step - loss: 0.0209 - acc: 0.9931
Epoch 42/100
50000/50000 [==============================] - 18s 363us/step - loss: 0.0276 - acc: 0.9908
Epoch 43/100
50000/50000 [==============================] - 18s 360us/step - loss: 0.0236 - acc: 0.9919
Epoch 44/100
50000/50000 [==============================] - 18s 359us/step - loss: 0.0255 - acc: 0.9912
Epoch 45/100
50000/50000 [==============================] - 18s 360us/step - 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
# input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
# classifier.predict(input_example)

loss, acc = classifier.evaluate(x_test, y_test)
print("Loss :     %f"%loss)
print("Accuracy : %f"%acc)

10000/10000 [==============================] - 3s 282us/step
Loss :     0.939218
Accuracy : 0.855900
